# The Dynamic Programming Approach to the Knapsack Problem in Julia

## The Knapsack Problem
Given a set of item $I$ , each item $i \in I$ characterized by:
* its weight $w_i$
* its value $v_i$  

and

* a capaciy $K$ for a knapsack

find the subset of items in $I$
* that has maximum value 
* does not exceed the capacity of the knapsack

## Dynamic Programming

Dynamic programming is:
* widely use optimization technique
* works for certain classes of problems very well (computational biology), but not for others.


Basic principles:
* divide and conquer
* bottom-up technique

Conventions needed for the dynamic programming approach to the knapsack problem:
* assume that $I = \{1,2,...,n\}$
* $O(k,j)$ denotes the optimal solution to the knapsack problem with capacity $k$ and items $[1..j]$

These conventions let us break down the problem into subproblems.

## Model
The dynamic programming model is a very similar to the greedy model, but has a slight change that lets us break it down into subproblems.

Maximize
$$\sum_{i \in 1..j}v_i x_i$$

subject to
$$\sum_{i \in 1..j} w_i x_i \leq k$$
$$x_i \in \{0,1\} (i \in 1..j)$$


## Why dynamic programming? 

Dynamic programming is much faster than the naive recursive solution. To see this, first consider the recursive, top-down approach:
* Assume we know how to solve $O(k,j-1)$ for all k in 0..K
* We want to solve O(k,j). We are just considering one more item, i.e., item j.
* If $w_j \leq k$ , there are two cases:
  * Either we do not select item j, then the best solution we can obtain is $O(k, k-1)$
  * Or we select item j and the best solution is $v_j + O(k-w_j,j-1)$
* In summary:
  * $O(k,j) = max(O(k,j-1), v_j + O(k-w_j, j-1))$ if $w_j \leq k$
  * $O(k,j) = O(k, j-1)$ otherwise
* Base case:
  * $O(k,0) = 0$ for all k

In [5]:
function O(k, j)
    if (j == 0)
        return 0
    elseif (I[j][2] <= k)
        return max(
                O(k,j-1), 
                I[j][1] + O(k-I[j][2], j-1)
            )
    else 
        return O(k,j-1)
    end
end

O (generic function with 1 method)

The problem with this approach is it makes $2^{|I|}$ function calls, as it recursively re-solves the same subproblems multiple times. 

## Visualizing Dynamic Programming 
Dynamic programming will allow us to conver the top-down recursive method into a bottom-up iterative method, saving the solutions of the sub-problems as we go in a matrix, and using those to solve subsequent sub-problems until we reach the final answer.

It is useful to use a table to visualize this approach. 

Consider a case where we have a list of only 3 items, $I$, and a knapsack of capacity 9, $K$:

In [3]:
I = [(5,4),
     (6,5),
     (3,2)]
K = 9

9

We need somewhere to store the answers to our subproblems. Let's initialize a matrix of size (I+1, K+1) to save the answers to our subproblems in.

In [57]:
M = zeros(Int8, length(I)+1, K+1)

4×10 Matrix{Int8}:
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0

Each row in this matrix corresponds to an item in the list. 
Each column in this matrix corresponds to a certain capacity. 

The first row corresponds to the base case for items (no item), and the first column corresponds to base case for capacity (0 capacity). That is why we it is an n+1 size matrix in each dimension. (Also, when we access our item weights and values, we will need to use n-1). 

The value of the cell will hold the answer the particular subproblem $O(k,j)$

Since we will be iterating over this matrix and using it's indices to access our weights and values, we need to split our items list $I$ into two vectors $v$ and $w$:

In [86]:
unzip(a) = map(x->getfield.(a, x), fieldnames(eltype(a)))
v, w = unzip(I)

([5, 6, 3], [4, 5, 2])

In [91]:
for (i, row) in enumerate(eachrow(M))
    for (j, col) in enumerate(eachcol(M))
        if i == 1 || j == 1
            M[i,j] = 0
        elseif w[i-1] > j - 1 # j - 1 because julia is 1-indexed.
            M[i,j] = M[i-1,j]
        else
            M[i,j] = maximum([
                    M[i-1,j],
                    v[i-1] + M[i-1,j - w[i-1]]
                    ])
        end
    end
end
M

4×10 Matrix{Int8}:
 0  0  0  0  0  0  0  0  0   0
 0  0  0  0  5  5  5  5  5   5
 0  0  0  0  5  6  6  6  6  11
 0  0  3  3  5  6  8  9  9  11

The total value of the optimal solution is found in the lower right hand corner of the table. The optimal solution can be found by comparing the last value in each row. Here we generate a vector $x_i$ of decision variabels:

In [117]:
j = size(M)[2]
x = zeros(Int8, size(M)[1]-1)
for i = 1:size(M)[1]-1
    if M[i,j] != M[i+1,j]
        x[i] = 1
    end
end
x

3-element Vector{Int8}:
 1
 1
 0

## Examples
Now that we have a complete dynamic programming approach to the knapsack problem, we can do a few exampes.

In [1]:
function dpksp(I, K)
    unzip(a) = map(x->getfield.(a, x), fieldnames(eltype(a)))
    v, w = unzip(I)
    n, k = length(I)+1, K+1
    
    M = zeros(Int, n, k)
    
    for i = 1:n
        for j = 1:k
            if i == 1 || j == 1
                M[i,j] = 0
            elseif w[i-1] > j - 1 # j - 1 because julia is 1-indexed.
                M[i,j] = M[i-1,j]
            else
                M[i,j] = maximum([
                        M[i-1,j],
                        v[i-1] + M[i-1,j - w[i-1]]
                        ])
            end
        end
    end
    
    x = zeros(Int, n-1)
    for i = 1:n-1
        if M[i,k] != M[i+1,k]
            x[i] = 1
        end
    end
    
    return M[n,k], x
end

dpksp (generic function with 1 method)

Here is our original setup just to confirm the new function works:

In [4]:
dpksp(I,K)

(11, [1, 1, 0])

Here is an additional problem where we know the maximum value is 44 already:

In [133]:
I2 = [(16,2),
      (19,3),
      (23,4),
      (28,5)]
K2 = 7
dpksp(I2,K2)

(Int8[1, 1, 1, 1], 44)

## Complexity of dynamic programming
The time complexity of this algorithm is the time is takes to fill the table, so $O(K*|I|)$, which is  O(n)! 

This is not polynomial however, because the storage is big (log(K) bits). It is a psuedo-polynomial algorithm, meaning it is polynomial for low K. 